# Settle Up Problem (Dlužníček)

## Motivation

You went on a trip with a group of your friends. All of you shared some expenses, and now it is the time to settle all the debts. It is clear that everyone should pay the same amount; however, people are lazy, and so you want to find the solution which minimizes the number of transactions.

## Input

You are given the following:

* A set of people $P$
* For every person $i \in P$ the cost $c_i$ (i.e., amount of money that $i$ payed)

For the experiments, you may use the following instance:

In [2]:
P = set(["A", "B", "C", "D"])
c = {"A": 0, "B": 590, "C": 110, "D": 300}  # c_i is accessed by calling c[i]
sv = sum(c.values())/len(c)  # the settlement value

Number $sv$ (the settlement value) is the fair price that every person should pay.

## Output

You should find a list of tuples $(x_k, y_k, z_k)$ representing the transactions: person $x_k$ should pay person $y_k$ z_k euros. The number of transactions (i.e., the length of the list) should be minimized.

For the given instance, the **optimal solution** consists of 3 transactions, namely

```
A -> B: 250.0
C -> D: 50.0
C -> B: 90.0
```

## Exercise

Implement the ILP model of the problem. You can assume that the settlement value is int (or was rounded).

In [7]:
from itertools import product as cartesian
list(cartesian(P, P))

[('C', 'C'),
 ('C', 'D'),
 ('C', 'A'),
 ('C', 'B'),
 ('D', 'C'),
 ('D', 'D'),
 ('D', 'A'),
 ('D', 'B'),
 ('A', 'C'),
 ('A', 'D'),
 ('A', 'A'),
 ('A', 'B'),
 ('B', 'C'),
 ('B', 'D'),
 ('B', 'A'),
 ('B', 'B')]

In [22]:
import gurobipy as g  # import Gurobi module
from gurobipy import GRB
from itertools import product as cartesian


# model --------------------------------------------------
m = g.Model()

# - ADD VARIABLES
F = m.addVars(cartesian(P, P), vtype=GRB.INTEGER, name='F', lb=0)
O = m.addVars(cartesian(P, P), vtype=GRB.BINARY, name='O')
#print(g.quicksum(T[('A', b)] for b in P))

# - ADD CONSTRAINTS
M = sum(c.values())

for p1 in P:
    m.addConstr(g.quicksum(F[(p1, p2)] for p2 in P) +
                c[p1] -
                g.quicksum(F[(p2, p1)] for p2 in P) == sv)
# TODO
for people in cartesian(P, P):
    m.addConstr(M*O[people] >= F[people])
    m.addConstr(F[people] >= O[people])

# - SET OBJECTIVE
# TODO
m.setObjective(O.sum())

# call the solver -------------------------------------------
m.optimize()

# print the solution -----------------------------------------
print('\nSOLUTION:')
print("\n".join([f'{p1} -> {p2}: {int(x.x)}' for ((p1, p2), x) in sorted(F.items(), key=lambda x: x[1].x, reverse=True) if x.x > 0]))


Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 36 rows, 32 columns and 88 nonzeros
Model fingerprint: 0x879c7564
Variable types: 0 continuous, 32 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 3e+02]
Found heuristic solution: objective 3.0000000
Presolve removed 8 rows and 8 columns
Presolve time: 0.00s
Presolved: 28 rows, 24 columns, 72 nonzeros
Variable types: 0 continuous, 24 integer (12 binary)

Root relaxation: objective 3.900000e-01, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.39000    0    3    3.00000    0.39000  87.0%     -    0s
     0     0     cutoff    0         3.00000    3.00000  0.00%     -    0s

Cutting planes:
  Gomory: 1
  MIR: 4
  Flow cover: 9
  Flow path: 1
 

## Additional experiments

* experiment with different values of the 'big M' constant
* try to generalize the model to work even with float sv (e.g., rounded to 0.01)

More comments can be found in `settle_up_models.ipynb.zip` from the Lab 3.
 